<a href="https://colab.research.google.com/github/JyzMinaBF/Retirement_Calculation/blob/main/Retirement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

life_male = pd.read_csv("https://raw.githubusercontent.com/JyzMinaBF/Miscellaneous/main/life_male.csv")
life_female = pd.read_csv("https://raw.githubusercontent.com/JyzMinaBF/Miscellaneous/main/life_female.csv")

In [ ]:
import numpy as np

def month_to_pay_change(year_in_pension, retire_age):
  if year_in_pension <= 15:
    month_to_pay = year_in_pension
  elif year_in_pension > 15:
    month_to_pay = np.min([2 * (year_in_pension - 15) + 15, 45])

  if month_to_pay == 45 and retire_age > 60:
    month_to_pay = np.min([month_to_pay + retire_age - 60, 50])

  return month_to_pay

def once_for_all_bef(fee_month_36, year_in_pension, retire_age):
  month_to_pay = month_to_pay_change(year_in_pension, retire_age)
  return fee_month_36 * month_to_pay

def once_for_all_aft(fee_month_60, year_in_pension, retire_age):
  month_to_pay = month_to_pay_change(year_in_pension, retire_age)
  return fee_month_60 * month_to_pay

def receive_cont(fee_month_60, year_in_pension, diff_of_age):
  st1 = fee_month_60 * year_in_pension * 0.00775 + 3000
  st2 = fee_month_60 * year_in_pension * 0.0155
  return np.max([st1, st2]) * (1 + 0.04 * diff_of_age)

def change(birth_year):
    if birth_year <= 46:
      return 60
    if birth_year in (47, 48, 49, 50):
      return birth_year + 14
    else:
      return 65

def discount(cf, r, year):
  n = int(round(12 * year, 0))
  pv = sum([round(cf / round((1 + r)**i, 20), 20) for i in range(1, n+1)])
  return pv

def throw_back(cf, r, pv_threshold):
  n = 0       # Initialize time period
  pv = 0      # Initialize present value

  while pv < pv_threshold:
    pv += cf / round((1 + r)**n, 20)
    n += 1

  return round(n/12, 3)

def range_calculated(year_from_105):
  period = year_from_105//9
  if year_from_105 <= 7:
    smallest = 26400
  else:
    smallest = 26400* (1.013 ** (year_from_105-7))
  biggest = 43900 * (1.045 ** period)
  diff = (biggest - smallest) / 12
  return [smallest + diff * i for i in range(13)]

def level_calculated(fee, birth_year, retire_age):
  ranges = range_calculated(birth_year + retire_age - 105)
  if fee <= ranges[0]:
    return ranges[0]
  elif fee > ranges[12]:
    return ranges[12]
  else:
    for i in range(len(ranges)-1):
      if fee > ranges[i] and fee <= ranges[i+1]:
        return ranges[i+1]

def condition_loop(fee_month_60, birth_year, retire_age, year_in_pension, diff_of_age, expected_inflation_rate, expected_life):
  diff_to_base = diff_of_age + 5
  base_age = retire_age - diff_to_base
  base_year_in_pension = year_in_pension - diff_to_base
  results = {}
  print("\n此外，以下為你在不同年齡退休下，如活至預期壽命時，折現至你現今預期退休年齡的金額：")
  for i in range(11):
    present_age = base_age + i
    present_year_in_pension = base_year_in_pension + i
    present_fee = level_calculated(fee_month_60, birth_year, present_age)
    month_money = receive_cont(present_fee, present_year_in_pension, i - 5)
    total_money = discount(month_money, expected_inflation_rate/12, expected_life - present_age) / ((1 + expected_inflation_rate) ** (i - diff_to_base))
    results[total_money] = present_age
    print("年齡：", present_age, " 可收：", "%.2f"%total_money)
  return "綜上所述，最佳的退休年齡是：" + str(results[max(results.keys())])



def pension(birth_year, year_in_pension, retire_age, sex, year_in_dang_job, fee_month_36, fee_month_60,
            longest_in_one_company, nation_year, expected_inflation_rate, expected_life, 民國九十八年以前曾投保):
  try:
    birth_year = int(birth_year)
    year_in_pension = float(year_in_pension)
    retire_age = int(retire_age)
    year_in_dang_job = int(year_in_dang_job)
    fee_month_36 = int(fee_month_36)
    fee_month_60 = int(fee_month_60)
    longest_in_one_company = int(longest_in_one_company)
    nation_year = int(nation_year)
    expected_inflation_rate = float(expected_inflation_rate)
    if expected_life == "0":
      if sex == "男":
        expected_life = retire_age + life_male.iloc[retire_age, 1]
      else:
        expected_life = retire_age + life_female[retire_age, 1]
    else:
      expected_life = int(expected_life)
  except:
    return "請輸入數字。"

  fee_month_36_changed = level_calculated(fee_month_36, birth_year, retire_age)
  fee_month_60_changed = level_calculated(fee_month_60, birth_year, retire_age)

  cond1, cond2, cond3 = False, False, False

  if 民國九十八年以前曾投保 == True:
    if sex == "女" and year_in_pension >= 1 and retire_age >= 55:
      cond1 = True
    elif sex == "男" and year_in_pension >= 1 and retire_age >= 60:
      cond1 = True
    elif year_in_pension >= 15 and retire_age >= 55:
      cond1 = True
    elif longest_in_one_company >= 25:
      cond1 = True
    elif year_in_pension >= 25 and retire_age >= 50:
      cond1 = True
    elif year_in_dang_job >= 5 and retire_age >= 55:
      cond1 = True

  limit_age = change(birth_year)
  diff_of_age = retire_age - limit_age

  if diff_of_age >= -5:
    if year_in_pension < 15:
      cond2 = True
    elif year_in_pension >= 15:
      cond3 = True

  if year_in_dang_job >= 15 and retire_age >= 55:
    cond3 = True
  elif (nation_year + year_in_pension) >= 15 and retire_age >= 65:
    cond3 = True

  print("勞保老年金可以選擇的請領方式：\n")
  if cond1:
    money_1 = once_for_all_bef(fee_month_36_changed, year_in_pension, retire_age)
    print("一次請領老年給付：可以")
    print("此方法可請領金額為：", "%.2f"%money_1, "\n")
  else:
    money_1 = 0
    print("一次請領老年給付：無法", "\n")

  if cond2:
    money_2 = once_for_all_aft(fee_month_60_changed, year_in_pension, retire_age)
    print("老年一次金給付：可以")
    print("此方法可請領金額為：", "%.2f"%money_2, "\n")
  else:
    money_2 = 0
    print("老年一次金給付：無法", "\n")

  if cond3:
    money_3 = receive_cont(fee_month_60_changed, year_in_pension, diff_of_age)
    print("老年年金給付：可以")
    print("每月可領取金額為：", "%.2f"%money_3)
    print("領取五年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 5))
    print("領取十年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 10))
    print("領取十五年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 15))
    print("領取二十年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 20))
    print("領取至您預期壽命的「", "%.2f"%expected_life, "」歲折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, expected_life - retire_age))
    if cond3 and (cond1 or cond2):
      print("此方法相較一次領之方法如再存活至:", retire_age + throw_back(money_3, expected_inflation_rate/12, np.max([money_1, money_2])), "歲，將可領得更多")
    print(condition_loop(fee_month_60, birth_year, retire_age, year_in_pension, diff_of_age, expected_inflation_rate, expected_life))
  else:
    money_3 = 0
    print("老年年金給付：無法")

  return max([money_1, money_2, discount(money_3, expected_inflation_rate/12, expected_life - retire_age)])

In [ ]:
def PMT(FV, n, IY):
  n = n * 12
  i = IY / 12
  PMT = round(FV * i / ((1 + i) ** n - 1), 2)
  return PMT

def retirement(現在年齡, 開始工作年齡, 起薪, 薪資成長率, 累積期投資報酬率 = None, 累積期通膨率 = 1.35, 性別 = "男",
               預計退休年齡 = None, 退休後投資報酬率 = None, 預計退休後平均餘命 = None, 預計退休後每月花費 = None, 現有存款 = None,
               自提比例 = 0, 定存比例 = 0, 美十年債券比例 = 0, BBB債券比例 = 0,  股票比例 = 0):
  age = int(現在年齡)
  start_work_age = int(開始工作年齡)
  wage_start = int(起薪)
  wage_inc = float(薪資成長率) / 100
  if 累積期投資報酬率 == "預設":
    ac_roi = 0
  else:
    ac_roi = float(累積期投資報酬率) /100
  ac_inf = float(累積期通膨率) / 100
  retire_age = int(預計退休年齡)
  if 退休後投資報酬率 == "預設":
    retire_roi = 0.009845
  else:
    retire_roi = float(退休後投資報酬率) / 100
  deposits = int(現有存款)
  自提比例 = float(自提比例)
  定存比例 = float(定存比例)
  美十年債券比例 = float(美十年債券比例)
  BBB債券比例 = float(BBB債券比例)
  股票比例 = float(股票比例)
  if 預計退休後平均餘命 == "預設":
    if 性別 == "男":
      expected_life = life_male.iloc[int(retire_age), 1]
    else:
      expected_life = life_female.iloc[int(retire_age), 1]
  else:
    expected_life = int(預計退休後平均餘命)

  year_in_pension = retire_age - start_work_age
  month_in_pension = year_in_pension * 12
  fee_month_36 = np.mean([wage_start * ((1 + wage_inc) ** (year_in_pension - i)) for i in range(1, 4)])
  fee_month_60 = np.mean([wage_start * ((1 + wage_inc) ** (year_in_pension - i)) for i in range(1, 6)])
  if 112 - age + start_work_age < 98:
    before_98 = True
  else:
    before_98 = False

  first_layer = pension(birth_year = 112 - age, year_in_pension = year_in_pension, retire_age = retire_age, sex = 性別, year_in_dang_job = 0,
                        fee_month_36 = fee_month_36, fee_month_60 = fee_month_60, longest_in_one_company = 0, nation_year = 0,
                        expected_inflation_rate = ac_inf, expected_life = expected_life + retire_age, 民國九十八年以前曾投保 = before_98)

  second_layer = 0

  for i in range(year_in_pension):
    second_layer = second_layer * (1 + max(ac_roi, 0.0144))
    second_layer += wage_start * ((1+wage_inc) ** i) * (0.06 + 自提比例) * 12
  print("\n勞退可領：", "%.2f"%second_layer)

  if any([定存比例, 美十年債券比例, BBB債券比例, 股票比例]):
    定存比例 += 1 - (定存比例 + 美十年債券比例 + BBB債券比例+ 股票比例)
    total_mean = (0.9845 * 定存比例 + 2.2614 * 美十年債券比例 + 3.926 * BBB債券比例 + 15.306 * 股票比例) / 100
    total_sd = (((0.2315 * 定存比例) ** 2) + ((0.7588 * 美十年債券比例) ** 2) + ((1.14 * BBB債券比例) ** 2) + ((13.126 * 股票比例) ** 2) +
                0.2315 * 定存比例 * 0.7588 * 美十年債券比例 * 0.718 + 0.7588 * 美十年債券比例 * 1.14 * BBB債券比例 * 0
                + 0.2315 * 定存比例 * 1.14 * BBB債券比例 * 0.512 + 1.14 * BBB債券比例 * 13.126 * 股票比例 * 0.362
                + 0.2315 * 定存比例 * 13.126 * 股票比例 * 0.431 + 0.7588 * 美十年債券比例 * 13.126 * 股票比例 * (-0.544)) ** (1/2) / 100
  else:
    total_mean = max(ac_roi, 0.0144)
  third_layer = deposits * (1 + total_mean) ** (retire_age - age)

  retire_discount_rate = 1 / (1 + (ac_inf - retire_roi)/12)
  if 預計退休後每月花費 == "預設":
    expense_after_retire_month = wage_start * ((1+wage_inc) ** (year_in_pension)) * 0.75
  else:
    expense_after_retire_month = int(預計退休後每月花費) * ((1+ac_inf) ** ((retire_age - age))) * expected_life * 12
  expense_after_retire = expense_after_retire_month * (retire_discount_rate ** (expected_life * 12) - 1) / (retire_discount_rate - 1)

  lack = expense_after_retire - first_layer - second_layer - third_layer
  print("\n退休後到退休時預計需要的總支出：", "%.2f"%expense_after_retire)
  print("資金的缺口有多大：", "%.2f"%lack)

  if lack < 0:
    print("恭喜！你無需再為退休做額外的規劃～")
  else:
    if any([定存比例, 美十年債券比例, BBB債券比例, 股票比例]):
      定存比例 += 1 - (定存比例 + 美十年債券比例 + BBB債券比例+ 股票比例)
      total_mean = (0.9845 * 定存比例 + 2.2614 * 美十年債券比例 + 3.926 * BBB債券比例 + 15.306 * 股票比例) / 100
      total_sd = (((0.2315 * 定存比例) ** 2) + ((0.7588 * 美十年債券比例) ** 2) + ((1.14 * BBB債券比例) ** 2) + ((13.126 * 股票比例) ** 2) +
                  0.2315 * 定存比例 * 0.7588 * 美十年債券比例 * 0.718 + 0.7588 * 美十年債券比例 * 1.14 * BBB債券比例 * 0
                  + 0.2315 * 定存比例 * 1.14 * BBB債券比例 * 0.512 + 1.14 * BBB債券比例 * 13.126 * 股票比例 * 0.362
                  + 0.2315 * 定存比例 * 13.126 * 股票比例 * 0.431 + 0.7588 * 美十年債券比例 * 13.126 * 股票比例 * (-0.544)) ** (1/2) / 100
      print("\n平均每月需要存的金額為:", PMT(lack, min(retire_age - age, retire_age - start_work_age), total_mean))
      print("95%的信心水準下，你每月最少要存的金額為：", PMT(lack, min(retire_age - age, retire_age - start_work_age), total_mean + 1.96 * total_sd))
      print("95%的信心水準下，你每月最多要存的金額為：", PMT(lack, min(retire_age - age, retire_age - start_work_age), total_mean - 1.96 * total_sd))
    else:
      total_mean = max(ac_roi, 0.0144)
      print("每月需要存的金額為:", PMT(lack, retire_age - age, total_mean))


In [ ]:
from ipywidgets import interact_manual
interact_manual(
    retirement,
    現在年齡 = "", 開始工作年齡 = "", 起薪 = "", 薪資成長率 = "",
    累積期投資報酬率 = "預設", 累積期通膨率 = "1.35",
    性別 = ["男", "女"],預計退休年齡 = "65", 退休後投資報酬率 = "預設",
    預計退休後平均餘命 = "預設", 預計退休後每月花費 = "預設", 現有存款 = "0",
    自提比例 = "0", 定存比例 = "0", 美十年債券比例 = "0", BBB債券比例 = "0", 股票比例 = "0"
);

interactive(children=(Text(value='', description='現在年齡'), Text(value='', description='開始工作年齡'), Text(value='',…

In [ ]:
interact_manual(
    retirement,
    現在年齡 = "25", 開始工作年齡 = "25", 起薪 = "35000", 薪資成長率 = "2.5",
    累積期投資報酬率 = "2", 累積期通膨率 = "2",
    性別 = ["男", "女"],預計退休年齡 = "65", 退休後投資報酬率 = "預設",
    預計退休後平均餘命 = "預設", 預計退休後每月花費 = "預設", 現有存款 = "0",
    自提比例 = "0", 定存比例 = "0.7", 美十年債券比例 = "0.1", BBB債券比例 = "0", 股票比例 = "0.2"
);

interactive(children=(Text(value='25', description='現在年齡'), Text(value='25', description='開始工作年齡'), Text(value…

In [ ]:
interact_manual(
    retirement,
    現在年齡 = "20", 開始工作年齡 = "25", 起薪 = "65000", 薪資成長率 = "3.5",
    累積期投資報酬率 = "預設", 累積期通膨率 = "1.35",
    性別 = ["男", "女"],預計退休年齡 = "60", 退休後投資報酬率 = "預設",
    預計退休後平均餘命 = "預設", 預計退休後每月花費 = "預設", 現有存款 = "0",
    自提比例 = "0", 定存比例 = "0.1", 美十年債券比例 = "0.1", BBB債券比例 = "0", 股票比例 = "0.8"
);

interactive(children=(Text(value='20', description='現在年齡'), Text(value='25', description='開始工作年齡'), Text(value…

In [ ]:
interact_manual(
    retirement,
    現在年齡 = "50", 開始工作年齡 = "20", 起薪 = "19000", 薪資成長率 = "3",
    累積期投資報酬率 = "預設", 累積期通膨率 = "1.35",
    性別 = ["男", "女"],預計退休年齡 = "65", 退休後投資報酬率 = "預設",
    預計退休後平均餘命 = "預設", 預計退休後每月花費 = "預設", 現有存款 = "1000000",
    自提比例 = "0", 定存比例 = "0.8", 美十年債券比例 = "0", BBB債券比例 = "0", 股票比例 = "0.2"
);

interactive(children=(Text(value='50', description='現在年齡'), Text(value='20', description='開始工作年齡'), Text(value…

In [ ]:
interact_manual(
    retirement,
    現在年齡 = "35", 開始工作年齡 = "25", 起薪 = "23000", 薪資成長率 = "3.5",
    累積期投資報酬率 = "預設", 累積期通膨率 = "1.5",
    性別 = ["男", "女"],預計退休年齡 = "60", 退休後投資報酬率 = "預設",
    預計退休後平均餘命 = "預設", 預計退休後每月花費 = "預設", 現有存款 = "350000",
    自提比例 = "0", 定存比例 = "0.3", 美十年債券比例 = "0", BBB債券比例 = "0.4", 股票比例 = "0.3"
);

interactive(children=(Text(value='35', description='現在年齡'), Text(value='25', description='開始工作年齡'), Text(value…